In [ ]:
# 急騰銘柄をリストアップしてその特徴を分析する
# 一週間とかで株価が2倍になるような銘柄

In [ ]:
from pathlib import Path
import datetime

import polars as pl

import stock

In [ ]:
code_list = stock.kabutan.get_code_list()
current_date = datetime.date.today()
start_date = current_date - datetime.timedelta(days=365)

In [ ]:
# 直近一年間の中で、数日で大きく値上がりした銘柄のリストを取得
target_dict = {}
for code in code_list:
    df = stock.kabutan.read_data_csv(code, start_date=start_date)
    df = df.with_columns(
        pl.col("close").rolling_max(window_size=5).shift(-5).interpolate("nearest").alias("maximum"),
        pl.col("close").rolling_min(window_size=5).alias("minimum"),
    )
    df = df.with_columns(
        (pl.col("maximum") > pl.col("minimum") * 1.5).alias("flag")
    )
    if len(df.filter(pl.col("flag") == True)) > 0:
        target_dict[code] = df.filter(pl.col("flag") == True)

In [ ]:
# 買い戦略を考える
## 逆指値で前日より高い場合は買い
## 損切りは前日安値 or 8%下の安い方

# 売却戦略を考える
## 50%上昇したら半分売却、20%下落したら売り、3週間以上経過したら売り


In [ ]:
def is_watch_list(code, current_date, with_df = False):
    start_date = current_date - datetime.timedelta(days=365)
    df = stock.kabutan.read_data_csv(code, start_date=start_date, end_date=current_date)
    fdf = (
        stock.kabutan.read_financial_csv(code)
        .filter(pl.col("annoounce_date") <= current_date)
        .sort(pl.col("annoounce_date"))
    )
    if len(fdf) > 0 and current_date - fdf["annoounce_date"][-1] < datetime.timedelta(days=1):
        #print("Near financial result announcement date")
        return False

    # 過去10日の値動きの大きさを計算
    window_size = 10
    avg_key = "avg{}".format(window_size)
    stddev_key = "stddev{}".format(window_size)
    df = df.with_columns(
        pl.col("close").rolling_mean(window_size=window_size).alias(avg_key),
        pl.col("close").rolling_std(window_size=window_size).alias(stddev_key),
    )

    # ギャップアップしている
    df = df.with_columns(
        (pl.col("close") > pl.col(avg_key) + pl.col(stddev_key)).alias("breakpoint")
    )

    # 出来高が増加（急増）
    df = df.with_columns(
        pl.col("volume").rolling_mean(window_size=window_size).shift().alias("max_volume")
    )
    df = df.with_columns((pl.col("volume") > pl.col("max_volume") * 2).alias("volume_increase"))

    if len(df) > 0 and df["breakpoint"][-1] and df["volume_increase"][-1]:
        # 高値で引けている
        flag = False
        # if (df["close"][-1] - df["low"][-1]) / max(df["high"][-1] - df["low"][-1], 1e-5) > 0.8:
        #     flag = True
        if df["close"][-1] >= df["open"][-1]:
            flag = True
        # 出来高が急増
        if df["volume"][-1] > df["max_volume"][-1] * 20:
            flag = True
        # if df["close"][-1] > df["open"][-1]: 
        # 小型株
        market_cap = stock.kabutan.data.get_market_capitalization(code)
        if flag and market_cap is not None and market_cap < 1000:
            return True
    
    if with_df:
        return False, df
    return False

In [ ]:
target_in_watchlist = {}
for code, df in target_dict.items():
    df = df.with_columns(
        pl.col("date").map_elements(lambda x: is_watch_list(code, x), return_dtype=bool).alias("is_watch_list")
    ).filter(pl.col("is_watch_list"))
    if len(df) > 0:
        target_in_watchlist[code] = df

In [ ]:
num_target = len(target_dict)
num_target_in_wl = len(target_in_watchlist)
print("num_target = {}, num_target_in_wl = {}, {}".format(num_target, num_target_in_wl, num_target_in_wl / num_target))

In [ ]:
pl.Config.set_tbl_cols(-1)
is_watch_list("4011", current_date=datetime.date(2024, 2, 14), with_df=True)

In [ ]:
for code, df in target_in_watchlist.items():
    print(code, df["date"].to_list())